In [16]:
# !pip install facenet-pytorch

In [115]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import albumentations as A
import os
from pathlib import Path


from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
from albumentations.pytorch import ToTensorV2

In [122]:
img_dir = Path('/opt/ml/input/data/eval/images')
seg_dir = Path('/opt/ml/input/data/eval/images_seg')

face_crop_dir = Path('/opt/ml/input/data/eval/images_face_crop')
seg_crop_dir = Path('/opt/ml/input/data/eval/images_seg_crop')
seg_crop_face_upper_dir = Path('/opt/ml/input/data/eval/images_seg_crop_face_upper')

if not os.path.isdir(face_crop_dir):
    os.mkdir(face_crop_dir)
    
if not os.path.isdir(seg_crop_dir):
    os.mkdir(seg_crop_dir)
    
if not os.path.isdir(seg_crop_face_upper_dir):
    os.mkdir(seg_crop_face_upper_dir)

csv_path = '/opt/ml/input/data/eval/info.csv'
df = pd.read_csv(csv_path)
df

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0


In [123]:
def faceCrop(img, faces, w_ratio, h_ratio, x, y):
    x1, y1, x2, y2 = faces[0][0]
    width = x2 - x1
    height = y2 - y1
    
    new_x = int(x1 - width * w_ratio / 2)
    new_y = int(y1 - height * h_ratio / 2)
    
    if(new_x < 0):
        w_ratio = x1 / width
        new_x = 0
    if(new_y < 0):
        h_ratio = y1 / height
        new_y = 0
    
    new_width = int(width * (1 + w_ratio))
    new_height = int(height * (1 + h_ratio))
    
    if(new_x + new_width > x):
        new_width = x - new_x
    if(new_y + new_height > y):
        new_height = y - new_y 
    
    transform = A.Compose([
                        A.Crop(new_x, new_y, new_x+new_width, new_y+new_height),
                        A.Resize(224, 224)
                        ])
    

    img = transform(image=np.array(img))['image']
    return img

train

In [118]:
# li = df["mask"].to_list()
# normal_idx = []
# for i in range(len(li)):
#     if li[i] == 2:
#         normal_idx.append(i)

In [124]:
detector = MTCNN()

In [125]:
# count = 0
# people_n = int(len(df)/7)

# for idx in range(1883, people_n):
#     ori_normal_path = Path(df['path'].iloc[normal_idx[idx]])
#     ori_normal_img = Image.open(ori_normal_path)
#     ori_normal_face = detector.detect(ori_normal_img)
#     img_fc_folder_dir = Path(str(ori_normal_path.parent).replace('images', 'images_face_crop'))
#     img_sc_folder_dir = Path(str(ori_normal_path.parent).replace('images', 'images_seg_crop'))
#     if not os.path.isdir(img_fc_folder_dir):
#         os.mkdir(img_fc_folder_dir)
    
#     if not os.path.isdir(img_sc_folder_dir):
#         os.mkdir(img_sc_folder_dir)
    
#     for j in range(7):
#         ori_path = Path(df['path'].iloc[idx*7+j])
#         seg_path = Path(str(df['path'].iloc[idx*7+j]).replace('images', 'images_seg'))
        
#         ori_img = Image.open(ori_path)
#         seg_img = Image.open(seg_path)
        
#         ori_face = detector.detect(ori_img)
#         if(ori_face[1][0] != None):
#             face_crop_img = Image.fromarray(faceCrop(ori_img, ori_face, 0.3, 0.4, 384, 512))
#             seg_crop_img = Image.fromarray(faceCrop(seg_img, ori_face, 0.3, 0.4, 384, 512))
#         else:
#             face_crop_img = Image.fromarray(faceCrop(ori_img, ori_normal_face, 0.3, 0.4, 384, 512))
#             seg_crop_img = Image.fromarray(faceCrop(seg_img, ori_normal_face, 0.3, 0.4, 384, 512))

        
#         face_crop_path = Path(str(df['path'].iloc[idx*7+j]).replace('images', 'images_face_crop'))
#         seg_crop_path = Path(str(df['path'].iloc[idx*7+j]).replace('images', 'images_seg_crop'))

#         face_crop_img.save(face_crop_path)
#         seg_crop_img.save(seg_crop_path)
        
#     if(idx % 100 == 0):
#         print(idx)

## eval

In [129]:
count = 0

transform = A.Compose([A.Crop(44, 44, 180, 180), A.Resize(224, 224)])
upper_transform = A.Compose([A.Crop(0, 0, 224, 112), A.Resize(224, 224)])
no_face = []
for idx in range(len(df)):

    ori_path = img_dir / Path(df['ImageID'].iloc[idx])
    seg_path = seg_dir / Path(df['ImageID'].iloc[idx])

    ori_img = Image.open(ori_path)
    seg_img = Image.open(seg_path)
    
    ori_face = detector.detect(ori_img)
    
    if(ori_face[1][0] != None):
        face_crop_img = Image.fromarray(faceCrop(ori_img, ori_face, 0.3, 0.4, 384, 512))
        seg_crop_img = Image.fromarray(faceCrop(seg_img, ori_face, 0.3, 0.4, 384, 512))
    else:
        face_crop_img = Image.fromarray(transform(image=np.array(ori_img))['image'])
        seg_crop_img = Image.fromarray(transform(image=np.array(seg_img))['image'])
        no_face.append(ori_path)

    seg_crop_upper_img = Image.fromarray(upper_transform(image=np.array(seg_crop_img))['image'])

    face_crop_path = face_crop_dir / Path(df['ImageID'].iloc[idx])
    seg_crop_path = seg_crop_dir / Path(df['ImageID'].iloc[idx])
    seg_crop_fu_path = seg_crop_face_upper_dir / Path(df['ImageID'].iloc[idx])

    face_crop_img.save(face_crop_path)
    seg_crop_img.save(seg_crop_path)
    seg_crop_upper_img.save(seg_crop_fu_path)

    if(idx % 1000 == 0):
        print(idx)

print(f'no faces -> {len(no_face)}')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
no faces -> 198
